In [1]:
'''检查EXIT-SEP数据一致性并合并基线表'''
#! 手动修改: 
# 6-终点指标-含住ICU时间+住院时间+临床结局+费用  删除第一行标头 D1, D28
# 2.2-D1D4D7-APACHEII评分-1817例.xlsx 表头手动改为 APACHE_II_D1, D4, D7
# 2.4-D1-D7感染性休克-1817例.xlsx 表头7个 D1感染休克-0无/1有 分别改为 D1, D2, D3... 感染休克-0无/1有
# 4.1 - 4.4: 67例未用药基线表头“结果”改成对应的指标名称
# 4.5-D1-D7血气分析-1817例.xlsx 删除表头D1-D7，只保留 D1 基线血气, 另存为 4.5-D1血气分析-1817例.xlsx; 此外有一例编号070007患者未在随机化表中出现，缺少070008号患者，确认070007是否为记录错误

import os, sys
import pandas as pd

# 检测运行环境
def in_notebook():
    return 'IPKernelApp' in getattr(globals().get('get_ipython', lambda: None)(), 'config', {})

if in_notebook():
    notebook_dir = os.getcwd()
    src_path = os.path.abspath(os.path.join(notebook_dir, '..'))
else:
    src_path = os.path.abspath(os.path.dirname(os.path.dirname(__file__)))
sys.path.append(src_path) if src_path not in sys.path else None

from src.utils import *
from src.setup import *
EXIT_SEP_PATH = r'E:\BaiduNetdiskWorkspace\My workspace\23.4.4-PhD-thesis\EXIT-SEP\EXIT-SEP病例数据-手工处理' # 试验原始数据


# 合并构建特征

In [2]:
filelist = getfiles(EXIT_SEP_PATH)

lab_sheet_dict = {'4.1': (['红细胞(RBC)', '血红蛋白(Hb)', '白细胞(WBC)', '中性粒细胞（N）', '淋巴细胞(L)', '血小板（PLT）', '红细胞压积（HCT）'], '67例未用药病例血常规基线'),
                  '4.2': (['谷丙转氨酶（ALT)', '谷草转氨酶(AST)', '总胆红素(STB)', '尿素氮（BUN）', '血肌酐（Cr）', '血糖（Glu）', '血钾（K）', '血钠（Na）'], '67例未用药病例血生化基线'),
                  '4.3': (['纤维蛋白原（Fg）', '凝血酶原时间（PT）', '部分活化凝血酶原时间（APTT）', 'D-二聚体（D-Dimer）'], '67例未用药病例凝血基线'),
                  '4.4': (['C反应蛋白（CRP）', '降钙素原（PCT）'], '67例未用药病例炎症指标基线')
                  }

df = None
for file in filelist:
    fname = os.path.split(file)[1]
    print(f'处理数据: {fname}')

    # 跳过字典
    if fname=='1.4-基线合并疾病归类.xlsx':
        continue
    
    # 跳过合并用药
    elif fname.startswith(('5','~')):
        continue

    # 病原学检查
    elif fname.startswith('1.2'):
        df_temp1 = pd.read_excel(file)

        df_temp = pd.read_excel(file, sheet_name='病原学检查结果')
        df_temp['Gram-_infect'] = df_temp['致病菌名称'].isin(range(1,13)).astype(int)
        df_temp['Gram+_infect'] = df_temp['致病菌名称'].isin(range(13,20)).astype(int)
        df_temp['Fungi_infect'] = df_temp['致病菌名称'].isin(range(20,29)).astype(int)
        df_temp['atypical_infect'] = (df_temp['致病菌名称']==29).astype(int)
        df_temp['virus_infect'] = df_temp['致病菌名称'].isin([30, 31]).astype(int)
        df_temp['other_infect'] = df_temp['致病菌名称'].isin([29, 30, 31]).astype(int)


        df_temp['Gram_neg_resist'] = (df_temp['耐药情况']=='阴性菌耐药').astype(int)
        df_temp['Gram_pos_resist'] = (df_temp['耐药情况']=='阳性菌耐药').astype(int)
        df_temp['Fungi_resist'] = (df_temp['耐药情况']=='真菌耐药').astype(int)
        df_temp['multidrug_resist'] = (df_temp['多重耐药']=='多重耐药').astype(int)

        df_temp = df_temp.groupby('受试者代码')[["Gram-_infect","Gram+_infect","Fungi_infect","atypical_infect","virus_infect","other_infect",
                                            "Gram_neg_resist","Gram_pos_resist","Fungi_resist","multidrug_resist"]].agg(max)
        df_temp = df_temp1.merge(df_temp, on='受试者代码', how='left', validate='1:1').rename(columns={'受试者代码':'ID'})
        df = df.merge(df_temp, on=['ID'], how='left', validate='1:1') if not (df is None) else df_temp

    # 生命体征
    elif fname.startswith('1.3'):
        df_temp = pd.read_excel(file)
        index_col = df_temp.columns[df_temp.columns.str.contains(r'受试者[编代]码')][0]
        df_temp = df_temp.rename(columns={index_col:'ID'})
        if '研究中心名称' in df_temp.columns:
            df_temp.drop(columns='研究中心名称', inplace=True)
        
        # 选择第一次测量
        mask_v1 = df_temp['入组天数'] == 1
        df_temp = df_temp[mask_v1]
        
        df = df.merge(df_temp, on=['ID'], how='left', validate='1:1') if not (df is None) else df_temp

    # 实验室检查数据处理
    elif fname[:3] in lab_sheet_dict.keys():
        
        (lab_sheet_list, non_treat_sheet) = lab_sheet_dict[fname[:3]]

        df_lab = None
        # 1750 例受试者数据
        for sheet in lab_sheet_list:
            df_temp = pd.read_excel(file, sheet_name=sheet)
            # 选择基线数据
            mask_v1 = df_temp['访视编号']=='sv1'
            df_temp = df_temp[mask_v1]

            # 选择检查结果
            index_col = df_temp.columns[df_temp.columns.str.contains(r'受试者[编代]码')][0]
            df_temp = df_temp[[index_col, '结果']].rename(columns={'结果':sheet, index_col:'ID'})

            if df_lab is None:
                df_lab = df_temp
            else:
                df_lab = df_lab.merge(df_temp, on=['ID'], how='left', validate='1:1')
        
        # 67例未用药基线
        df_non_treat = pd.read_excel(file, sheet_name=non_treat_sheet)
        # 选择检查结果, lab_sheet_list (检查指标子表)已手动填写到67例未用药基线表检查结果的对应列名
        index_col = df_non_treat.columns[df_non_treat.columns.str.contains(r'受试者[编代]码')][0]
        df_non_treat = df_non_treat[[index_col, *lab_sheet_list]].rename(columns={index_col:'ID'})

        # 合并到 1750例数据
        df_lab = pd.concat([df_lab, df_non_treat])
        
        # 与其他数据合并
        df = df.merge(df_lab, on=['ID'], how='left', validate='1:1') if not (df is None) else df_lab

    else:
        df_temp = pd.read_excel(file)
        index_col = df_temp.columns[df_temp.columns.str.contains(r'受试者[编代]码')][0]
        df_temp = df_temp.rename(columns={index_col:'ID'})
        if '研究中心名称' in df_temp.columns:
            df_temp.drop(columns='研究中心名称', inplace=True)

        df = df.merge(df_temp, on=['ID'], how='left', validate='1:1') if not (df is None) else df_temp

df.to_excel(f'{DATA}/EXIT_SEP_merged.xlsx', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\BaiduNetdiskWorkspace\\My workspace\\23.4.4-PhD-thesis\\EXIT-SEP\\EXIT-SEP病例数据-手工处理'

# 选择特征数据

In [ ]:
SELECTED_VAR = ['ID',
                '试验组别',
                '年龄_x',
                '性别_x',
                '民族',
                '身高',
                '体重',
                'BMI',
                '有无吸烟史',
                '吸烟指数', # 吸烟指数=每日吸烟支数×吸烟年数
                '有无酗酒史',
                '感染来源',
                '感染部位',
                '病原学检查（已做=2，未做=1）',
                '整体有无耐药情况',
                "Gram-_infect","Gram+_infect","Fungi_infect",
                "Gram_neg_resist","Gram_pos_resist","Fungi_resist","multidrug_resist",
                '体温(℃)',
                '心率(次/分)',
                '呼吸(次/分)',
                '收缩压(mmHg)',
                '舒张压(mmHg)',
                '呼吸功能[PaO2/FIO2(mmHg) SaO2/FIO2]', # SOFA
                '凝血指标[血小板109/L]', # SOFA
                '肝脏[胆红素(μmol/L)]', # SOFA
                '心血管[低血压]', # SOFA
                '中枢神经系统[Glasgow昏迷分数]', # SOFA
                '肾脏[肌酐(μmol/L)或尿量(mL/d)]', # SOFA
                'SOFA总分_y',
                'APACHE_II_D1',
                'D1-DIC总分',
                'D1感染休克-0无/1有',
                'D1平均动脉压（MAP）mmHg',
                'D1血乳酸（Lac）mmol/l',
                'D1是否使用呼吸支持-0未用/1使用',
                'D1呼吸支持类型', # 机械通气:有创/无创; 其他为非机械通气:面罩吸氧/湿化雾疗仪/高流量湿化氧疗仪
                'D1是否使用CCRT-0未用/1使用',
                'D1是否使用营养支持-0未用/1使用',
                'D1营养支持途径',

                '红细胞(RBC)', '血红蛋白(Hb)', '白细胞(WBC)', '中性粒细胞（N）', '淋巴细胞(L)', '血小板（PLT）', '红细胞压积（HCT）', # 血常规
                '谷丙转氨酶（ALT)', '谷草转氨酶(AST)', '总胆红素(STB)', '尿素氮（BUN）', '血肌酐（Cr）', '血糖（Glu）', '血钾（K）', '血钠（Na）', # 血生化
                '纤维蛋白原（Fg）', '凝血酶原时间（PT）', '部分活化凝血酶原时间（APTT）', 'D-二聚体（D-Dimer）', # 凝血
                'C反应蛋白（CRP）', '降钙素原（PCT）', # 炎症
                'PH', '氧合指数（PaO2/FiO2）mmHG', '碳酸氢（HCO3-）mmol/l', '氧分压（PaO2）mmHg', '血乳酸（Lac）mmol/l','二氧化碳分压（PaCO2）mmHg', # 血气

                '28天预后', # 主要结局- 28天死亡
                '死亡时间', # 主要结局- 28天死亡时间 (生存分析用,surv_time)
                '放弃治疗在院外死亡的，按院内死亡记录',# 次要结局- 院内死亡
                '计算住ICU天数时，死亡的按住满28天计算', # 次要结局-住ICU天数
                '计算住院天数时，死亡的按住满28天计算', # 次要结局-住院天数
                'D1-D7有无感染休克-0无/1有', # 次要结局-是否进展性发生septic shock
                ]

df_selected = df[SELECTED_VAR].copy()
df_selected.to_excel(f'{DATA}/EXIT_SEP_selected.xlsx', index=False)

# 数据清洗

In [ ]:
pd.set_option('display.max_columns', 100)
df_selected

In [26]:
df_clean = df_selected[['ID']].copy()
df_clean['XBJ_intervention'] = df_selected['试验组别'].replace({'试验药物':1, '对照药物':0})
# 排除标准特征对齐
# df_clean['metastatic_cancer'] = 0
# df_clean['hematologic_cancer'] = 0
# df_clean['AIDS'] = 0
df_clean['age'] = df_selected['年龄_x']
df_clean['sex'] = df_selected['性别_x'].replace({'男':1, '女':0})
# 民族，身高，体重
df_clean['BMI'] = df_selected['BMI']
# df_clean['smoke_status'] = df_selected['有无吸烟史'] # 1 有; 0 无
# df_clean['smoke_pack_year'] = (df_selected['吸烟指数'] / 20).fillna(0)  #! 有杂入文字
# df_clean['drink_status'] = df_selected['有无吸烟史'] # 1 有; 0 无
df_clean['primary_infection_site_lung'] = (df_selected['感染来源']=='肺部感染').astype(int)
df_clean['primary_infection_site_abdo'] = (df_selected['感染来源']=='腹腔感染').astype(int)
df_clean['primary_infection_site_uri'] = (df_selected['感染部位'].str.contains('泌尿')).astype(int)
df_clean['primary_infection_site_skin'] = (df_selected['感染部位'].str.contains('皮肤')).astype(int)
df_clean['primary_infection_site_brain'] = (df_selected['感染部位'].str.contains('颅内')).astype(int)
df_clean['primary_infection_site_blood'] = (df_selected['感染部位'].str.contains('血液|败血症')).astype(int)

df_clean['pathogen_test'] = df_selected['病原学检查（已做=2，未做=1）'].replace({2:0, 1:1}) # 0-已做; 1-未做， 用作缺失指示器
pathogen_list = ['Gram-_infect','Gram+_infect', 'Fungi_infect', # 感染
                'Gram_neg_resist','Gram_pos_resist','Fungi_resist','multidrug_resist'] # 耐药
df_clean[pathogen_list] = df_selected[pathogen_list].fillna(0) # 已清洗病原学数据直接复制，0代表缺失（未做病原学检查）

df_clean['temperature'] = df_selected['体温(℃)'].astype(float)
df_clean['heart_rate'] = df_selected['心率(次/分)'].astype(float)
df_clean['respir_rate'] = df_selected['呼吸(次/分)'].astype(float)
df_clean['SBP'] = df_selected['收缩压(mmHg)'].astype(float)
df_clean['DBP'] = df_selected['舒张压(mmHg)'].astype(float)
df_clean['SOFA_respiration'] = df_selected['呼吸功能[PaO2/FIO2(mmHg) SaO2/FIO2]'].astype(float)
df_clean['SOFA_coagulation'] = df_selected['凝血指标[血小板109/L]'].astype(float)
df_clean['SOFA_liver'] = df_selected['肝脏[胆红素(μmol/L)]'].astype(float)
df_clean['SOFA_cardio'] = df_selected['心血管[低血压]'].astype(float)
df_clean['SOFA_cns'] = df_selected['中枢神经系统[Glasgow昏迷分数]'].astype(float)
df_clean['SOFA_renal'] = df_selected['肾脏[肌酐(μmol/L)或尿量(mL/d)]'].astype(float)
df_clean['SOFA'] = df_selected['SOFA总分_y'].astype(float)
df_clean['APACHE_II'] = df_selected['APACHE_II_D1'].astype(float)
df_clean['DIC-score'] = df_selected['D1-DIC总分'].astype(float)
df_clean['septic_shock'] = df_selected['D1感染休克-0无/1有'].astype(int)
df_clean['MAP'] = df_selected['D1平均动脉压（MAP）mmHg'].astype(float)
df_clean['Lac'] = df_selected['D1血乳酸（Lac）mmol/l'].astype(float)

df_clean['Respiratory_Support'] = df_selected['D1是否使用呼吸支持-0未用/1使用'].astype(int)
mask_oxy = df_selected['D1呼吸支持类型'].str.contains('氧疗|吸氧', na=False)
mask_NIPPV = (df_selected['D1呼吸支持类型'] == '无创')
mask_MV = df_selected['D1呼吸支持类型'].str.contains('有创', na=False)
df_clean.loc[mask_oxy, 'Respiratory_Support'] = 1 # 非侵入、非机械通气
df_clean.loc[mask_NIPPV, 'Respiratory_Support'] = 2 # 无创正压通气(Non-invasive Positive Pressure Ventilation, NIPPV)
df_clean.loc[mask_MV, 'Respiratory_Support'] = 3 # 有创机械通气 (Mechanical Ventilation, MV)
df_clean['MV'] = df_selected['D1呼吸支持类型'].str.contains('有创', na=False).astype(int)
df_clean['NIPPV'] = df_selected['D1呼吸支持类型'].str.contains('无创', na=False).astype(int)
df_clean['MV/NIPPV'] = df_clean[['MV','NIPPV']].max(axis=1)

df_clean['CCRT'] = df_selected['D1是否使用CCRT-0未用/1使用'].astype(int) # Continuous Renal Replacement Therapy 持续性肾脏替代治疗

df_clean['nutri_support'] = df_selected['D1是否使用营养支持-0未用/1使用'].astype(int)
df_clean['nutri_support_enteral'] = df_selected['D1营养支持途径'].str.contains('肠内', na=False)
df_clean['nutri_support_parenteral'] = df_selected['D1营养支持途径'].str.contains('肠外', na=False)
# (df_clean['nutri_support_enteral'] + df_clean['nutri_support_parenteral'] == df_clean['nutri_support']).all() # 肠内/肠外均不使用时即为无营养支持

df_clean[['RBC','Hb','WBC', 'NE%', 'LYM%', 'PLT', 'HCT',
          'ALT', 'AST', 'STB', 'BUN', 'Scr', 'Glu', 'K+', 'Na+',
          'Fg', 'PT', 'APTT', 'D-Dimer',
          'CRP', 'PCT', 
          'PH', 'PaO2', 'PaO2/FiO2', 'PaCO2', 'HCO3-', 'Lac', 
          ]] = df_selected[['红细胞(RBC)', '血红蛋白(Hb)', '白细胞(WBC)', '中性粒细胞（N）', '淋巴细胞(L)', '血小板（PLT）', '红细胞压积（HCT）', # 血常规
                            '谷丙转氨酶（ALT)', '谷草转氨酶(AST)', '总胆红素(STB)', '尿素氮（BUN）', '血肌酐（Cr）', '血糖（Glu）', '血钾（K）', '血钠（Na）', # 血生化
                            '纤维蛋白原（Fg）', '凝血酶原时间（PT）', '部分活化凝血酶原时间（APTT）', 'D-二聚体（D-Dimer）', # 凝血
                            'C反应蛋白（CRP）', '降钙素原（PCT）', # 炎症
                            'PH', '氧合指数（PaO2/FiO2）mmHG', '碳酸氢（HCO3-）mmol/l', '氧分压（PaO2）mmHg', '血乳酸（Lac）mmol/l','二氧化碳分压（PaCO2）mmHg', # 血气
                            ]].replace(['ND','NK', 'NA', 'na'], np.nan)\
                                .applymap(lambda x: re.sub("＞|>|＜|<", '', x) if isinstance(x, str) else x)\
                                .astype(float)


df_clean['in_hospital_mortality'] = (df_selected['放弃治疗在院外死亡的，按院内死亡记录'].isin(['ICU', '其他科室', '院内'])).astype(int) # 0-未发生院内死亡 1-院内死亡
df_clean['28d_mortality'] = df_selected['28天预后'].replace({'存活':0, '死亡':1, ' ':np.nan}) # 0-存活 1-死亡
df_clean['7d_septic_shock'] = df_selected['D1-D7有无感染休克-0无/1有'].astype(float)

In [ ]:
# 校验结局
mask_xbj = df_clean['XBJ_intervention'] == 1
mask_placebo = df_clean['XBJ_intervention'] == 0
mask_in_hos_death = (df_clean['in_hospital_mortality']==1)
mask_28d_death = (df_clean['28d_mortality']==1)

print(f'28天死亡: {mask_28d_death.sum()}({mask_28d_death.sum()/len(df_clean)*100:.2f}%); placebo:{(mask_placebo & mask_28d_death).sum()}; XBJ:{(mask_xbj & mask_28d_death).sum()}')
print(f'住院死亡: {mask_in_hos_death.sum()}({mask_in_hos_death.sum()/len(df_clean)*100:.2f}%); placebo:{(mask_placebo & mask_in_hos_death).sum()}; XBJ:{(mask_xbj & mask_in_hos_death).sum()}')

df_clean.to_csv(f'{DATA}/EXIT_SEP_clean.tsv.gz', sep='\t', compression='gzip', index=False)

In [ ]:
df_clean_worse_case_both_die = df_clean.copy()
df_clean_worse_case_xbj_die = df_clean.copy()

# 假设缺失结局时两组都在28天死亡（只对主要结局指标分析）
df_clean_worse_case_both_die['28d_mortality'] = df_clean_worse_case_both_die['28d_mortality'].fillna(1)

mask_28d_death = (df_clean_worse_case_both_die['28d_mortality']==1)
print('假设缺失结局时两组都死亡')
print(f'28天死亡: {mask_28d_death.sum()}; placebo:{(mask_placebo & mask_28d_death).sum()}; XBJ:{(mask_xbj & mask_28d_death).sum()}')


# 假设缺失结局时XBJ组在28天死亡，对照组存活
mask_missing_28d = df_clean_worse_case_xbj_die['28d_mortality'].isna()
df_clean_worse_case_xbj_die.loc[(mask_xbj & mask_missing_28d), '28d_mortality'] = 1
df_clean_worse_case_xbj_die.loc[(mask_placebo & mask_missing_28d), '28d_mortality'] = 0

mask_28d_death = (df_clean_worse_case_xbj_die['28d_mortality']==1)
print('假设缺失结局时XBJ死亡,对照组存活')
print(f'28天死亡: {mask_28d_death.sum()}; placebo:{(mask_placebo & mask_28d_death).sum()}; XBJ:{(mask_xbj & mask_28d_death).sum()}')

df_clean_worse_case_both_die.to_csv(f'{DATA}/EXIT_SEP_worse_case_both_die.tsv.gz', sep='\t', compression='gzip', index=False)
df_clean_worse_case_xbj_die.to_csv(f'{DATA}/EXIT_SEP_worse_case_xbj_die.tsv.gz', sep='\t', compression='gzip', index=False)